In [1]:
%env CUDA_VISIBLE_DEVICES=6

import math

import torch
from linear import HiggsLinear

env: CUDA_VISIBLE_DEVICES=6


In [2]:
SIZE = 4096
HADAMARD_SIZE = 1024

layer = HiggsLinear(
    SIZE, 1,
    2, 256,
    device="cuda",
    dtype=torch.float16,
)

In [3]:
input = torch.rand(1, 1, SIZE, device="cuda", dtype=torch.float16)

layer(input)

tensor([[[30.7656]]], device='cuda:0', dtype=torch.float16)

In [4]:
from aqlm.utils import unpack_int_data, _dequantize_weight

In [5]:
@torch.inference_mode()
def unpack_int_data(data: torch.IntTensor, nbits: int) -> torch.IntTensor:
    return data.to(torch.int64) % (2**nbits)

codebooks = torch.load("../grids/EDEN2-256.pt").half()

weight = _dequantize_weight(
    unpack_int_data(layer.codes, 8)[:,:,None],
    codebooks[None,:,None,:],
    # scales=layer.scales,
).reshape(1, SIZE//HADAMARD_SIZE, HADAMARD_SIZE) * layer.scales[...,None]

weight = weight.reshape(1, SIZE)

In [6]:
from fast_hadamard_transform import hadamard_transform

hadamard_transform(input.reshape(-1, HADAMARD_SIZE), scale=1/math.sqrt(HADAMARD_SIZE)).reshape(1, -1) @ weight.T

tensor([[30.7344]], device='cuda:0', dtype=torch.float16)